In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
df = pd.read_csv('stroke_data.csv').dropna()
df[:10]

In [ ]:
X = df.drop(columns=['stroke'])
y = df[['stroke']]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)

train_ds = TensorDataset(torch.tensor(X_train.values, dtype=torch.float32), torch.tensor(y_train.values, dtype=torch.float32))
test_ds = TensorDataset(torch.tensor(X_test.values, dtype=torch.float32), torch.tensor(y_test.values, dtype=torch.float32))

train_dl = DataLoader(train_ds, batch_size=20, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=20, shuffle=True)

In [ ]:
class TunableModel(nn.Module):
    def __init__(self, num_hidden_nodes, num_hidden_layers):
        super().__init__()

        # add Input Layer
        self.input_layer = nn.Linear(10, num_hidden_nodes)

        # Add a Mechanism that would add n number of hidden layers
        hidden_layers = []
        for i in range(num_hidden_layers):
            hidden_layers.append(nn.Linear(num_hidden_nodes, num_hidden_nodes))
            hidden_layers.append(nn.ReLU())
        
        # Add a Sequential Portion to house all hidden layers
        self.hidden_layers = nn.Sequential(*hidden_layers)

        # Add Output Layer
        self.output_layer = nn.Linear(num_hidden_nodes, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.input_layer(x)
        x = self.hidden_layers(x)
        x = self.output_layer(x)
        pred = self.sigmoid(x)
        return pred



In [ ]:
# Helper Functions
# Training Steps
def train(model, dataloader, epochs, loss_fn, optimizer, verbose=True):
    model.train()

    for epoch in range(epochs):
        i = 0
        for X_val, y_val in dataloader:
            pred = model(X_val)
            loss = loss_fn(pred, y_val)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            i += 1

            # Print Tracker only every 100 steps
            if i % 100 == 0 and verbose:
                print('Epoch {0} Step {1}: Loss - {2}'.format(epoch+1, i, loss.item()), end='\r')
    
    return model



def test(model, dataloader):
    model.eval()

    preds = []
    trues = []
    # Make Predictions
    with torch.no_grad():
        for X_val, y_val in dataloader:
            pred = model(X_val).round()
            preds.extend(pred.numpy())
            trues.extend(y_val.numpy())
    
    return f1_score(trues, preds)


# Define our optimization loop

def score_hyperparameter_sets(hyperparameter_sets):
    scores = []

    # Iterate through the entire set to get scores
    for hyperparameter_set in hyperparameter_sets:
        # Set Random State for Reproducibility
        torch.manual_seed(0)

        # Unpack all hyperparameters in the set
        hidden_nodes = hyperparameter_set['hidden_nodes']
        hidden_layers = hyperparameter_set['hidden_layers']
        # Define the Model with hyperparameters
        model = TunableModel(num_hidden_nodes=hidden_nodes, num_hidden_layers=hidden_layers)

        # Define Training Objects
        loss_fn = nn.BCELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
        
        # Train Model
        print(f'Training Model with HP: (Hidden Nodes-{hidden_nodes}, Hidden Layers-{hidden_layers})')
        model = train(model, train_dl, 10, loss_fn, optimizer)

        # Test Model
        score = test(model, test_dl)

        scores.append(score)


    # Make Table to show the results
    results = pd.DataFrame(hyperparameter_sets)
    results['score'] = scores

    return results

In [ ]:
hyperparameter_space = {
    'hidden_nodes': range(20, 200+1, 20),
    'hidden_layers': range(0, 5, 1)
}
hyperparameter_space

In [ ]:
def convert_hp_space_to_sets(hpo_space):
    hp_sets = []
    for hidden_nodes in hpo_space['hidden_nodes']:
        for hidden_layers in hpo_space['hidden_layers']:
            hp_sets.append({'hidden_nodes': hidden_nodes, 'hidden_layers': hidden_layers})
    
    return hp_sets

hp_sets = convert_hp_space_to_sets(hyperparameter_space)
hp_sets

In [ ]:
results_df = score_hyperparameter_sets(hp_sets)
results_df.sort_values(by='score', ascending=False)
